In [4]:
import tensorflow as tf

[Loading tensors from csv](https://www.tensorflow.org/versions/r0.9/how_tos/reading_data/index.html#file-formats)

In [5]:
filename_queue = tf.train.string_input_producer(["./files/train.csv"])

In [6]:
reader = tf.TextLineReader()

In [7]:
key, value = reader.read(filename_queue)

In [8]:
# Default values, in case of empty columns
record_defaults = [["ID"], ["NA"], ["2014-01-01 00:00:00"], [""], [""], 
                   ["Dog"], ["Unknown"], ["1 year"], ["NA"], ["NA"]]

In [9]:
animal_id, name, date_time, outcome_type, outcome_subtype, animal_type, sex_upon_outcome, age_upon_outcome, breed, color = tf.decode_csv(
        value, record_defaults=record_defaults)

##### Features Definition

The following attrs will be ignored:

- AnimalID: for obvious reasons.
- Name: we will need a need to quantify this before we can consider it as a feature (e.g., ranking based on name popularity, or "sounds like" popular name).
- DateTime: it appears to represent the date at which the outcome was recorded. Unless we can correlate that to external events that could potentially correlate with adoption rates (e.g., economy downturns, employment rates, etc.), this attribute won't provide much value in predicting future outcomes. In fact, it could potentially add noise to the model.
- OutcomeType: this is what we want to predict.
- OutcomeSubtype: may be another predictable state.

The following attrs will be included:

- AnimalType -> Convert to enum
- SexUponOutcome -> Convert to enum
- AgeUponOutcome -> Convert to float
- Breed -> Convert to enum
- Color -> Convert to enum

In [10]:
feature_attrs = [animal_type, sex_upon_outcome, age_upon_outcome, breed, color]
features = tf.pack(feature_attrs)

##### Possible predictions

The idea is to get a probability distribution of each possible outcome.

- OutcomeType:
    - Return_to_owner
    - Euthanasia
    - Adoption
    - Transfer
    - Died

In [11]:
num_possible_outcomes = 5

##### Variables

In [12]:
x = tf.placeholder(tf.float32, [None, len(feature_attrs)]) # maybe not float32 ... may need different types for each feature
W = tf.Variable(tf.zeros([len(feature_attrs), num_possible_outcomes]))
b = tf.Variable(tf.zeros([num_possible_outcomes]))
y_ = tf.placeholder(tf.float32, [None, num_possible_outcomes])

##### Model

In [13]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

##### Cost function

In [14]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

##### Training

In [15]:
learning_rate = 0.5
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

##### Running the calculations

In [16]:
epochs = 100 # increase eventually
batch_size = 100
init = tf.initialize_all_variables();
sess = tf.Session()
sess.run(init)
# TODO Figure out how to load batches from train data
# for i in range(epochs):
#     batch_xs, batch_ys = 